In [1]:
import sys
sys.path.insert(0, '/home/laura/Documents/stage2018')
from utils import *

In [9]:
a = np.random.rand(3, 2)
b = np.random.rand(4, 2)

In [10]:
a = np.hstack((a, np.array([[1], [2], [2]])))
a

array([[0.75196557, 0.27046303, 1.        ],
       [0.15803349, 0.42504083, 2.        ],
       [0.52854706, 0.62704325, 2.        ]])

In [13]:
b = np.hstack((b, np.array([[1], [1], [2], [1]])))
b

array([[0.29828396, 0.80840269, 1.        ],
       [0.92027047, 0.12301267, 1.        ],
       [0.55144727, 0.94063892, 2.        ],
       [0.37904276, 0.48622241, 1.        ]])

In [16]:
set1 = LabeledSet(2)
set2 = LabeledSet(2)
set1.addExamples(a[:,:2], a[:,2:])
set2.addExamples(b[:,:2], b[:,2:])

In [21]:
set3 = LabeledSet(2)
set3.addExamples(set1.x, set1.y)
set3.addExamples(set2.x, set2.y)
set3.x

array([[0.75196557, 0.27046303],
       [0.15803349, 0.42504083],
       [0.52854706, 0.62704325],
       [0.29828396, 0.80840269],
       [0.92027047, 0.12301267],
       [0.55144727, 0.94063892],
       [0.37904276, 0.48622241]])

In [22]:
set3.y

array([[1.],
       [2.],
       [2.],
       [1.],
       [1.],
       [2.],
       [1.]])

In [2]:
f = Dsr()
g = Log()
h = Sum()

rsdm = Gdm(h, g, f)

In [9]:
def discretize(H, labeled_set, a_j):
    '''
        H : discrimation measure
        labeled_set : labeled set
        a_j : attribute to discretize
        return threshold which minimizes the entropy function H for given feature a_j, 
            along with minimum entropy value
    '''
    
    n = labeled_set.size()
    ind = np.argsort(labeled_set.x[:,a_j],axis=0) # sort values according to attribute a_j
    
    # binary set : for each object w_i taken in ascending order of a_j value,
    # a_j(w_h) = 0 if a_j(w_h) <= a_j(w_i), 1 otherwise
    binary_set = LabeledSet(labeled_set.getInputDimension())
    binary_set.nb_examples = labeled_set.size()
    binary_set.x = np.ones(labeled_set.size())
    binary_set.y = labeled_set.y
    
    thresholds = []
    H_values = []
        
    dsa = np.ones((n, n))
    dsl = H.f.dominant_sets_label(binary_set)
    esa = np.ones((n, n))
    esl = H.f.equal_sets_label(binary_set)
    
    last = 0
    
    for i in range(n-1):
        current = labeled_set.getX(ind[i])[a_j]
        current_label = labeled_set.getY(ind[i])
        lookahead = labeled_set.getX(ind[i+1])[a_j]
        lookahead_label = labeled_set.getY(ind[i+1])
        binary_set.x[ind[i]] = 0
        
        if current == lookahead or current_label == lookahead_label:
            continue
        else:
            a = np.zeros((n,))
            a[ind[:i+1]] = 1
            dsa[ind[last:i+1]] = np.ones((n,))
            esa[ind[:i+1]] = a
            
            a = np.zeros((n,))
            a[ind[i+1:]] = 1
            
            dsa[ind[i+1:]] = a
            esa[ind[i+1:]] = a

        thresholds.append((current + lookahead) / 2.0)
        H_values.append(H.value(binary_set, a_j, dsa, dsl, esa, esl))
        last = ind[i+1]
      
    min_entropy = min(H_values)
    min_threshold = thresholds[np.argmin(H_values)]
    return (min_threshold, min_entropy)

In [10]:
start = time.time()
set2, t = generate_2Ddataset(0, 2, 1000, 0.6, 0.1, [[-10, 10], [-10, 10]])

t1, h1 = discretize(rsdm, set2, 0)

print("running time : ", format(time.time() - start))

running time :  22.1731960773468
